In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [ ]:
DATA_PATH = kagglehub.competition_download('csiro-biomass')

print('Data source import complete.')

In [ ]:
# !pip install torch==2.6.0 -qq
# !pip install torchvision==0.21.0 -qq
# !pip install timm==1.0.19 -qq
# !pip install albumentations==2.0.8 -qq

In [ ]:
import os, random, math
from dataclasses import dataclass
from typing import Dict, List, Optional

import numpy as np
import pandas as pd
from PIL import Image

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import GroupKFold, StratifiedGroupKFold

import timm
import torchvision
from torchvision import transforms

In [ ]:
print(pd.__version__)
print(np.__version__)
print(torch.__version__)
print(torchvision.__version__)
print(timm.__version__)
print(A.__version__)

In [ ]:
train_dir = os.path.join(DATA_PATH, 'train')
test_dir = os.path.join(DATA_PATH, 'test')

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
train_df.head(10)

In [ ]:
for col in train_df.columns:
    print(col, train_df[col].nunique())

In [ ]:
train_df.Species.value_counts()

In [ ]:
train_df.State.value_counts()

In [ ]:
len(train_df)

In [ ]:
train_df['Sampling_Date'] = pd.to_datetime(train_df['Sampling_Date'])
train_df.Sampling_Date.max(), train_df.Sampling_Date.min()

In [ ]:
train_df['Year'] = train_df['Sampling_Date'].dt.year
train_df['Month'] = train_df['Sampling_Date'].dt.month

train_df.Year.value_counts(), train_df.Month.value_counts()

In [ ]:
aa = train_df[train_df.State=="NSW"]["Month"]
aa.value_counts()

In [ ]:
train_df["SeasonQ"] = train_df["Sampling_Date"].dt.to_period("Q").astype(str)
train_df["SeasonQ"].value_counts()

In [ ]:
from dataclasses import dataclass, asdict

In [ ]:
@dataclass
class CFG:
    CSV_PATH: str = DATA_PATH + "/train.csv"
    IMG_ROOT: str = DATA_PATH + ""  # keep as you had
    MODEL_NAME: str = "vit_base_patch14_dinov2.lvd142m"  # DINOv2 in timm
    IMG_SIZE: int = 518
    N_FOLDS: int = 5
    EPOCHS: int = 40
    BATCH_SIZE: int = 32
    NUM_WORKERS: int = 4

    LR_HEAD: float = 6e-4
    LR_BACKBONE: float = 6e-5
    # LR_HEAD: float = 1.2e-3
    # LR_BACKBONE: float = 1.2e-4
    WEIGHT_DECAY: float = 0.05

    SEED: int = 42
    DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

    SAVE_DIR: str = "model_outputs"

    USE_TILING: bool = True
    TILES_TRAIN: Optional[int] = 2
    TILES_VAL: Optional[int] = None

    FREEZE_EPOCHS: int = 3
    UNFREEZE_LAST_N_BLOCKS: int = 2

    TARGET_ORDER = ["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"]
    LOSS_WEIGHTS = {
        "Dry_Green_g": 0.1,
        "Dry_Dead_g": 0.1,
        "Dry_Clover_g": 0.1,
        "GDM_g": 0.2,
        "Dry_Total_g": 0.5
    }

    EMA_ENABLED: bool = True
    EMA_DECAY: float = 0.99
    EMA_WARMUP_STEPS: int = 200
    EMA_ON_CPU: bool = False

cfg_inst = CFG()

In [ ]:
def seed_everything(seed: int = CFG.SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def worker_init_fn(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def split_1x2_lr(img):
    h, w = img.shape[:2]
    w2 = w // 2
    return [img[:, :w2], img[:, w2:]]

def split_2x1_tb(img):
    h, w = img.shape[:2]
    h2 = h // 2
    return [img[:h2, :], img[h2:, :]]

In [ ]:
from copy import deepcopy

class ModelEMA:

    def __init__(self, model: nn.Module, decay: float = 0.999, device: str | None = None, warmup_steps: int = 0):
        self.ema = deepcopy(model).eval()
        for p in self.ema.parameters():
            p.requires_grad_(False)

        self.decay = decay
        self.warmup_steps = warmup_steps
        self.num_updates = 0

        if device is not None:
            self.ema.to(device)

    @torch.no_grad()
    def update(self, model: nn.Module):
        self.num_updates += 1

        if self.warmup_steps and self.num_updates < self.warmup_steps:
            d = 1.0 - (1.0 - self.decay) * (self.num_updates / self.warmup_steps)
        else:
            d = self.decay

        msd = model.state_dict()
        esd = self.ema.state_dict()

        for k, v in esd.items():
            mv = msd[k].detach()
            if v.dtype.is_floating_point:
                v.mul_(d).add_(mv, alpha=(1.0 - d))
            else:
                v.copy_(mv)


In [ ]:
class PastureDataset(Dataset):
    def __init__(self, df_wide: pd.DataFrame, transforms_img=None, use_tiling=True, tiles_train: Optional[int]=2,
                 split_mode: str = "tb"):
        self.df = df_wide.reset_index(drop=True)
        self.transforms = transforms_img
        self.use_tiling = use_tiling
        self.tiles_train = tiles_train
        self.split_mode = split_mode  # "tb" or "lr"

    def __len__(self):
        return int(len(self.df))

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        img_path = os.path.join(CFG.IMG_ROOT, row["image_path"])

        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        targets = np.array([row[t] for t in CFG.TARGET_ORDER], dtype=np.float32)
        y = torch.tensor(np.log1p(targets), dtype=torch.float32)

        if not self.use_tiling:
            if self.transforms:
                image = self.transforms(image=image)["image"]
            return image, y

        if self.split_mode == "tb":
            tiles = split_2x1_tb(image)
        elif self.split_mode == "lr":
            tiles = split_1x2_lr(image)
        else:
            raise ValueError("split_mode must be 'tb' or 'lr'")

        tiles_total = len(tiles)

        if self.tiles_train is not None and self.tiles_train < tiles_total:
            idxs = np.random.choice(tiles_total, self.tiles_train, replace=False)
            tiles = [tiles[i] for i in idxs]

        tile_tensors = []
        for timg in tiles:
            if self.transforms:
                timg = self.transforms(image=timg)["image"]
            else:
                timg = torch.from_numpy(timg).permute(2, 0, 1).float()
            tile_tensors.append(timg)

        x = torch.stack(tile_tensors, dim=0)
        return x, y


def build_transforms(train: bool = True):
    if train:
        return A.Compose([

            A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),

            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=180, p=0.7),

            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            # A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=20, val_shift_limit=10, p=0.4),

            A.RandomSnow(p=0.1),
            A.RandomShadow(p=0.1),
            A.RandomFog(p=0.1),
            A.RandomRain(p=0.1),
            A.RandomSunFlare(p=0.1),
            # A.RandomGravel(p=0.1),

            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.2),

            A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            A.ToTensorV2(),
        ],
      seed=42)
    else:
        return A.Compose([
            A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
            A.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
            A.ToTensorV2(),
        ],
      seed=42)


class DinoV2TileRegressor(nn.Module):
    def __init__(self, model_name=CFG.MODEL_NAME, out_dim=5, dropout=0.2, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0, dynamic_img_size=True)
        d = self.backbone.num_features
        self.drop = nn.Dropout(dropout)
        self.head = nn.Linear(d, out_dim)

    def forward(self, x):
        if x.dim() == 4:
            feat = self.backbone(x)
            return self.head(self.drop(feat))

        B, T, C, H, W = x.shape
        x = x.reshape(B*T, C, H, W)
        feat = self.backbone(x).reshape(B, T, -1).mean(dim=1)
        return self.head(self.drop(feat))


class WeightedMSE(nn.Module):
    def __init__(self, target_order: List[str], weights: Dict[str, float]):
        super().__init__()
        self.idx = {t:i for i,t in enumerate(target_order)}
        self.w = torch.tensor([weights[t] for t in target_order], dtype=torch.float32)

    def forward(self, pred: torch.Tensor, y: torch.Tensor):
        diff2 = (pred - y) ** 2
        w = self.w.to(pred.device)
        return (diff2 * w).mean()

class WeightedHuberLoss(nn.Module):
    def __init__(self, target_order, weights, delta=1.0):
        super().__init__()
        self.weights = torch.tensor([weights[t] for t in target_order]).cuda()
        self.huber = nn.HuberLoss(reduction='none', delta=delta)

    def forward(self, pred, y):
        loss = self.huber(pred, y)
        return (loss * self.weights).mean()


def r2_per_target(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:

    out = {}
    for i, t in enumerate(CFG.TARGET_ORDER):
        yt = y_true[:, i]
        yp = y_pred[:, i]
        ss_res = np.sum((yt - yp) ** 2)
        ss_tot = np.sum((yt - yt.mean()) ** 2) + 1e-12
        out[t] = 1.0 - (ss_res / ss_tot)
    return out


def weighted_r2(r2_dict: Dict[str, float]) -> float:
    return sum(CFG.LOSS_WEIGHTS[t] * r2_dict[t] for t in CFG.TARGET_ORDER)


def global_weighted_r2(y_true_mat: np.ndarray,
                       y_pred_mat: np.ndarray,
                       weights: Dict[str, float],
                       order: List[str]) -> float:

    w_cols = np.array([weights[t] for t in order], dtype=np.float64)
    W = np.broadcast_to(w_cols, y_true_mat.shape).astype(np.float64)

    y_true = y_true_mat.astype(np.float64).ravel()
    y_pred = y_pred_mat.astype(np.float64).ravel()
    w_flat = W.ravel()

    y_bar = np.sum(w_flat * y_true) / np.sum(w_flat)

    ss_res = np.sum(w_flat * (y_true - y_pred) ** 2)
    ss_tot = np.sum(w_flat * (y_true - y_bar) ** 2) + 1e-12

    return 1.0 - (ss_res / ss_tot)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, scheduler=None, ema: ModelEMA | None = None):
    model.train()
    running_loss = 0.0

    for images, targets in loader:
        images = images.to(CFG.DEVICE, non_blocking=True)
        targets = targets.to(CFG.DEVICE, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        preds = model(images)
        loss = criterion(preds, targets)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        if ema is not None:
            ema.update(model)

        if scheduler is not None:
            scheduler.step()

        running_loss += loss.item() * targets.size(0)

    return running_loss / len(loader.dataset)


@torch.no_grad()
def validate(model, loader, return_arrays: bool = False):
    model.eval()
    all_preds, all_tgts = [], []

    for images, targets in loader:
        images = images.to(CFG.DEVICE, non_blocking=True)
        preds = model(images)

        preds = torch.expm1(preds)
        targets = torch.expm1(targets)

        all_preds.append(preds.cpu().numpy())
        all_tgts.append(targets.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_tgts  = np.concatenate(all_tgts, axis=0)

    gw_r2 = global_weighted_r2(all_tgts, all_preds, CFG.LOSS_WEIGHTS, CFG.TARGET_ORDER)
    r2_dict = r2_per_target(all_tgts, all_preds)

    if return_arrays:
        return gw_r2, r2_dict, all_preds, all_tgts
    return gw_r2, r2_dict

In [ ]:
def freeze_backbone(model: DinoV2TileRegressor):
    for p in model.backbone.parameters():
        p.requires_grad = False

def unfreeze_last_blocks(model: DinoV2TileRegressor, last_n: int = 2):
    freeze_backbone(model)
    if hasattr(model.backbone, "blocks"):
        for b in model.backbone.blocks[-last_n:]:
            for p in b.parameters():
                p.requires_grad = True
    for p in model.head.parameters():
        p.requires_grad = True
    for p in model.drop.parameters():
        p.requires_grad = True

def build_optimizer(model: DinoV2TileRegressor, lr_head: float, lr_backbone: float, wd: float):
    param_groups = []
    bb_params = [p for p in model.backbone.parameters() if p.requires_grad]
    if bb_params:
        param_groups.append({"params": bb_params, "lr": lr_backbone})
    param_groups.append({"params": model.head.parameters(), "lr": lr_head})
    return torch.optim.AdamW(param_groups, weight_decay=wd)

In [ ]:
def run_fold(fold_id: int, df_wide: pd.DataFrame):
    print(f"\n========== Fold {fold_id + 1} / {CFG.N_FOLDS} ==========")

    trn_df = df_wide[df_wide["fold"] != fold_id].reset_index(drop=True)
    val_df = df_wide[df_wide["fold"] == fold_id].reset_index(drop=True)

    train_ds = PastureDataset(
        trn_df, transforms_img=build_transforms(train=True),
        use_tiling=CFG.USE_TILING, tiles_train=CFG.TILES_TRAIN, split_mode="lr"
    )
    val_ds = PastureDataset(
        val_df, transforms_img=build_transforms(train=False),
        use_tiling=CFG.USE_TILING, tiles_train=CFG.TILES_VAL, split_mode="lr"
    )

    g = torch.Generator(); g.manual_seed(CFG.SEED)

    train_loader = DataLoader(
        train_ds, batch_size=CFG.BATCH_SIZE, shuffle=True,
        num_workers=CFG.NUM_WORKERS, pin_memory=True, drop_last=True,
        worker_init_fn=worker_init_fn, generator=g
    )
    val_loader = DataLoader(
        val_ds, batch_size=CFG.BATCH_SIZE * 2, shuffle=False,
        num_workers=CFG.NUM_WORKERS, pin_memory=True,
        worker_init_fn=worker_init_fn, generator=g
    )

    model = DinoV2TileRegressor(model_name=CFG.MODEL_NAME, out_dim=len(CFG.TARGET_ORDER), pretrained=True).to(CFG.DEVICE)

    ema = None
    if getattr(CFG, "EMA_ENABLED", False):
        ema_device = "cpu" if getattr(CFG, "EMA_ON_CPU", False) else CFG.DEVICE
        ema = ModelEMA(model, decay=CFG.EMA_DECAY, device=ema_device, warmup_steps=getattr(CFG, "EMA_WARMUP_STEPS", 0))

    criterion = WeightedHuberLoss(CFG.TARGET_ORDER, CFG.LOSS_WEIGHTS, delta=1.0)

    freeze_backbone(model)
    optimizer = build_optimizer(model, lr_head=CFG.LR_HEAD, lr_backbone=0.0, wd=CFG.WEIGHT_DECAY)
    total_steps = CFG.EPOCHS * len(train_loader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-6)

    best_wr2 = -1e9

    for epoch in range(1, CFG.EPOCHS + 1):

        if epoch == CFG.FREEZE_EPOCHS + 1:
            unfreeze_last_blocks(model, last_n=CFG.UNFREEZE_LAST_N_BLOCKS)
            optimizer = build_optimizer(model, lr_head=CFG.LR_HEAD * 0.7, lr_backbone=CFG.LR_BACKBONE, wd=CFG.WEIGHT_DECAY)
            remaining_steps = (CFG.EPOCHS - epoch + 1) * len(train_loader)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max(1, remaining_steps), eta_min=1e-6)

        tr_loss = train_one_epoch(model, train_loader, optimizer, criterion, scheduler, ema=ema)

        eval_model = ema.ema.to(CFG.DEVICE) if ema is not None else model
        wr2, r2d = validate(eval_model, val_loader)

        best_wr2 = max(best_wr2, wr2)
        print(
            f"Epoch {epoch:02d}/{CFG.EPOCHS} | train_loss={tr_loss:.4f} | "
            f"val_gwR2={wr2:.4f} | " +
            ", ".join([f"{k}:{v:.3f}" for k, v in r2d.items()])
        )

    eval_model = ema.ema.to(CFG.DEVICE) if ema is not None else model
    fold_wr2, fold_r2d, fold_preds, fold_tgts = validate(eval_model, val_loader, return_arrays=True)
    print(f"[Fold {fold_id + 1}] Final val global weighted R² = {fold_wr2:.4f}")

    os.makedirs(CFG.SAVE_DIR, exist_ok=True)
    final_path = os.path.join(CFG.SAVE_DIR, f"fold{fold_id}_final.pth")

    payload = {
        "model": model.state_dict(),
        "cfg": asdict(cfg_inst),
        "fold": fold_id + 1,
    }
    if ema is not None:
        payload["ema"] = ema.ema.state_dict()

    torch.save(payload, final_path)
    print(f"Saved final checkpoint: {final_path}")

    return fold_preds, fold_tgts, fold_wr2


In [ ]:
train_df.head()

In [ ]:

# def make_folds(df: pd.DataFrame) -> pd.DataFrame:

#     df = df.copy()
#     df["Sampling_Date"] = pd.to_datetime(df["Sampling_Date"], errors="coerce")

#     # df = df.sort_values("image_path").reset_index(drop=True)

#     meta = (
#         df
#         .drop_duplicates("image_path")
#         [["image_path", "State", "Sampling_Date"]]
#         .copy().reset_index(drop=True)
#     )

#     meta["date_group"] = meta["Sampling_Date"].astype(str)
#     meta["State"] = meta["State"].astype(str)

#     sgkf = StratifiedGroupKFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
#     meta["fold"] = -1

#     for fold, (_, val_idx) in enumerate(sgkf.split(meta, y=meta["State"], groups=meta["date_group"])):
#         meta.loc[val_idx, "fold"] = fold

#     wide = df.pivot(index="image_path", columns="target_name", values="target").reset_index()

#     wide = wide[["image_path"] + CFG.TARGET_ORDER]
#     wide = wide.merge(meta[["image_path", "fold", "State", "date_group"]],
#                       on="image_path", how="left")
#     return wide


def make_folds(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    df["Sampling_Date"] = pd.to_datetime(df["Sampling_Date"], errors="coerce")

    meta = (
        df
        .drop_duplicates("image_path")
        [["image_path", "State", "Sampling_Date", "Height_Ave_cm", "Pre_GSHH_NDVI"]]
        .copy()
        .reset_index(drop=True)
    )

    wide = df.pivot(index="image_path", columns="target_name", values="target").reset_index()

    wide = wide.merge(meta, on="image_path", how="left")

    wide["day"] = wide["Sampling_Date"].dt.day
    wide["month"] = wide["Sampling_Date"].dt.month

    wide["group_key"] = (
        wide["day"].astype(str) + '/' +
        wide["month"].astype(str) + '_' +
        wide["State"].astype(str)
    )

    clover_present = (wide["Dry_Clover_g"].fillna(0) > 0).astype(int).astype(str)
    dead_present   = (wide["Dry_Dead_g"].fillna(0) > 0).astype(int).astype(str)

    wide["stratify_col"] = clover_present + '_' + dead_present
    sgkf = StratifiedGroupKFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
    wide["fold"] = -1

    for fold, (_, val_idx) in enumerate(sgkf.split(wide, y=wide["stratify_col"], groups=wide["group_key"])):
        wide.loc[val_idx, "fold"] = fold

    cols_to_keep = ["image_path"] + CFG.TARGET_ORDER + ["fold", "State", "Height_Ave_cm", "Pre_GSHH_NDVI", "group_key"]
    return wide[cols_to_keep]

In [ ]:
gseed_everything(CFG.SEED)

train = pd.read_csv(CFG.CSV_PATH)
train_wide = make_folds(train)

print(f'DF VALUE COUNTS: {train_wide["fold"].value_counts()}')
for f in sorted(train_wide["fold"].unique()):
    val = train_wide[train_wide["fold"] == f]
    trn = train_wide[train_wide["fold"] != f]
    print(f"Fold {f}: val={len(val)}, train={len(trn)}, sum={len(val)+len(trn)}")

oof_preds_list, oof_tgts_list, fold_gwr2_list = [], [], []

print("\n=== Training ===\n")
for fold in range(CFG.N_FOLDS):
    preds, tgts, fold_gwr2 = run_fold(fold, train_wide)
    oof_preds_list.append(preds)
    oof_tgts_list.append(tgts)
    fold_gwr2_list.append(fold_gwr2)
    print(f"Fold {fold} weighted R² = {fold_gwr2:.4f}")

OOF_PRED = np.concatenate(oof_preds_list, axis=0)
OOF_TRUE = np.concatenate(oof_tgts_list, axis=0)

oof_gw_r2 = global_weighted_r2(OOF_TRUE, OOF_PRED, CFG.LOSS_WEIGHTS, CFG.TARGET_ORDER)
oof_r2 = r2_per_target(OOF_TRUE, OOF_PRED)
mean_wr2 = float(np.mean(fold_gwr2_list))

print(f"\nAverage weighted R²: {mean_wr2:.4f}")
print(f"\nOverall OOF Global Weighted R²: {oof_gw_r2:.4f}\n")
print("Per-target OOF R²:")
for k, v in oof_r2.items():
    print(f"{k}: {v:.4f}")

In [ ]:
from google.colab import files

!zip -r model_outputs.zip /content/model_outputs
files.download('model_outputs.zip')